# Example of VRO-Rad

In [1]:
import argparse
import pickle
import pandas as pd
from radgraph import F1RadGraph
import numpy as np
def remove_ith_sentence(samples, i):
    updated_samples = []
    for sample in samples:
        sentences = sample.split('. ')  
        if 0 <= i < len(sentences):
            del sentences[i]  
        updated_sample = '. '.join(sentences)  
        updated_samples.append(updated_sample)
    return updated_samples

def count_consistent_entities_batch(hypothesis_annotation_lists_batch, reference_annotation_lists_batch):
    consistent_ratios = []
    for i in range(len(hypothesis_annotation_lists_batch)):
        hypothesis_entities = hypothesis_annotation_lists_batch[i]['entities']
        reference_entities = reference_annotation_lists_batch[i]['entities']

        reference_set = list((entity['tokens'], entity['label']) for entity in reference_entities.values()) # ignore the relation
        hypothesis_set = list((entity['tokens'], entity['label']) for entity in hypothesis_entities.values()) 

        consistent_count = 0
        for entity in hypothesis_entities.values():
            if (entity['tokens'], entity['label']) in reference_set:
                consistent_count += 1
        if len(hypothesis_set) > 0:
            ratio = consistent_count / len(hypothesis_set)
        else:
            ratio = float('nan') # no entity parsed within the sentence
        consistent_ratios.append(ratio)
    return consistent_ratios


/projectnb/neurips/.conda/envs/nlp_/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projectnb/neurips/.conda/envs/nlp_/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [21]:
# Load 5 reference (original) reports
with open("prediction_5.pkl", "rb") as f:
    pred = pickle.load(f)
# Load 10 sampled (hypothesis) reports for each of the 5 original reports 
with open("samples_5.pkl", "rb") as f:
    samples = pickle.load(f)

In [22]:
mode = "complete"
f1radgraph = F1RadGraph(reward_level=mode)

model_type not provided, defaulting to radgraph-xl


In [31]:
greedy = pred
all_sent = []
for i in range(len(greedy)):
    u_cent = []
    for sent in greedy[i].split('. '):
        batch_sents = [sent] * 10
        mean_reward, reward_list, hypothesis_annotation_lists, reference_annotation_lists = f1radgraph(hyps=batch_sents, refs=samples[i])
        conf = count_consistent_entities_batch(hypothesis_annotation_lists, reference_annotation_lists)
        u_cent.append([1 - i for i in conf])
    output = [np.mean(item) for item in u_cent]
    print(f"For sample {i} the uncertainty for each sentence is: {output}")
    all_sent.append(output)

For sample 0 the uncertainty for each sentence is: [0.95, 0.4, 0.2, 0.6, 0.7166666666666667, 0.9]
For sample 1 the uncertainty for each sentence is: [0.9, 0.7625, 0.7000000000000001, 0.9, 0.2, 0.9]
For sample 2 the uncertainty for each sentence is: [0.8, 0.7875, 0.7666666666666667, 0.75, 0.5, 0.95, 0.8]
For sample 3 the uncertainty for each sentence is: [0.525, 0.1, 0.775, 0.7]
For sample 4 the uncertainty for each sentence is: [nan, 0.52, 0.5, 0.2666666666666667, 0.95]
